In [8]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [9]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

In [10]:
#imports
import os
import time
import pandas as pd
import numpy as np
import sys
from io import StringIO
import glob
if not hasattr(sys, "version_info") or sys.version_info < (3, 5):
  raise SystemExit("This program requires Python 3.5 or later.")
import csv
import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
## https://github.com/ipython/ipyparallel
import ipyparallel as ipp
from IPython.display import JSON
import json
import statistics as st
from pympler import asizeof
from hurry.filesize import size
import tabletext
import sqlite3
import shutil

In [11]:
def mstime():
    import time
    return round(time.time() * 1000)

def sync():
    import os
    os.system('sync')
    
def touch(path):
    import os
    with open(path, 'a'):
        os.utime(path, None)
    sync()
    
def unic(l: list) -> list :
    return list(set(l))

def cleanupTmp():
    print("clean tmp in %s" % cutPathes(tmpFolder))
    if tmpFolder == "/tmp":
        print("invalid tmp folder!")
        exit(-1)
    for filename in os.listdir(tmpFolder):
        file_path = os.path.join(tmpFolder, filename)
        # print("remove %s" % file_path)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (cutPathes(file_path), e))

In [12]:
# read env
envHome = os.environ.get('HOME')
envRamHome = os.environ.get('RAMCASHE')

#init env
tmpFolder = "%s/tmp" % envRamHome
if os.path.isdir(tmpFolder):
    shutil.rmtree(tmpFolder, ignore_errors=True)

os.mkdir(tmpFolder)

In [13]:
data_sources = glob.glob("data/*_shops.xml")
plt.rcParams['figure.figsize'] = [30/2.54, 20/2.54]

# tags
client = ipp.Client()
workers = client[:]

midDb = "%s/midData/mid.db" % os.getcwd()
midDbExample = "%s/midData/mid.example.db" % os.getcwd()
midDbTmp = "%s/mid.%d.db" % (envRamHome, mstime())
baseDir = os.getcwd() + "/"

def cutPathes(s: str) -> str:
    if s.startswith(baseDir):
        return s[len(baseDir)::]
    if s.startswith(envRamHome):
        return s[len(envRamHome)::]
    return s

print("engines count: %d" % len(client.ids))
print("source files count: %d" % len(data_sources))
print(tmpFolder)

engines count: 4
source files count: 2751
/home/ed/ramcashe/tmp


In [14]:
workers["tmpFolder"] = tmpFolder
workers["dbquery_cratetable"] = '''
CREATE TABLE "ttids" (
	"rowid"	INTEGER NOT NULL UNIQUE,
	"shapshotid"	TEXT NOT NULL,
	"ttid"	INTEGER NOT NULL,
	"buy_list"	TEXT NOT NULL,
	"sell_list"	TEXT NOT NULL,
	"capacity"	INTEGER NOT NULL,
	"buystat_avg"	REAL NOT NULL,
	"buystat_min"	REAL NOT NULL,
	"buystat_max"	REAL NOT NULL,
	"buystat_median"	REAL NOT NULL,
	"buystat_gap"	REAL NOT NULL,
	"buystat_stdev"	REAL NOT NULL,
	"sellstat_avg"	REAL NOT NULL,
	"sellstat_min"	REAL NOT NULL,
	"sellstat_max"	REAL NOT NULL,
	"sellstat_median"	REAL NOT NULL,
	"sellstat_gap"	REAL NOT NULL,
	"sellstat_stdev"	REAL NOT NULL,
	PRIMARY KEY("rowid" AUTOINCREMENT)
);'''
dbquery_cratetable = workers["dbquery_cratetable"][0]

workers["dbquery_insert"] = '''
INSERT INTO 'ttids'(
"shapshotid",
"ttid",
"buy_list",
"sell_list",
"capacity",
"buystat_avg",
"buystat_min",
"buystat_max",
"buystat_median",
"buystat_gap",
"buystat_stdev",
"sellstat_avg",
"sellstat_min",
"sellstat_max",
"sellstat_median",
"sellstat_gap",
"sellstat_stdev"
) VALUES (
?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'''
dbquery_insert = workers["dbquery_insert"][0]

In [15]:
def processSnapshot(fname):
    import xml.etree.ElementTree as ET
    import statistics as st
    import sqlite3
    import os
    import time
    from pathlib import Path

    def mstime():
        return round(time.time() * 1000)
    def sync():
        os.system('sync')
    def touch(path):
        with open(path, 'a'):
            os.utime(path, None)
        sync()
    
    log = []
    p = Path(fname)
    baseDir = str(p.parent)
    dbName = "%s/%s.%s.db" % (tmpFolder, str(p.name), mstime())
    log.append("process %s. base: %s, tmp database: %s" % (fname, baseDir, dbName))
    def processShop(tree, time):
        shopData = []
        ttidList = []
        for good in tree:
            goodData = {}
            goodData["ttid"] = int(good.findall("ttid")[0].text)
            ttidList.append(goodData["ttid"])
            goodData["time"] = time
            goodData["shop"] = tree.attrib["name"]
            goodData["shopid"] = tree.attrib["id"]
            goodData["shopmoney"] = tree.attrib["money"]
            sellElems = good.findall("sell")
            if sellElems:
                goodData["sell"] = float(sellElems[0].text)
            else:
                goodData["sell"] = 0.0

            buyElems = good.findall("buy")
            if buyElems:
                goodData["buy"] = float(buyElems[0].text)
            else:
                goodData["buy"] = 0.0
            goodData["count"] = int(good.findall("count")[0].text)
            goodData["thingname"] = good.findall("thingname")[0].text
            shopData.append(goodData)
        return (shopData, ttidList)
    

    if os.path.exists(dbName):
        os.remove(dbName)
    sync()
    touch(dbName)
    timetag = "lastupdatetime"
    shopTag = "shop"
    xmltree = ET.parse(fname)
    xmlroot = xmltree.getroot()
    timeelem = xmlroot.find(timetag)
    dataTime = int(timeelem.text)
    orders = []
    ttids = []
    con = sqlite3.connect(dbName)    
    cur = con.cursor()
    q = dbquery_cratetable.replace("\n", " ")
    cur.execute(q)
    for shop in xmlroot.findall(shopTag):
        rc = processShop(shop, dataTime)
        shopData = rc[0]
        ttidList = rc[1]
        orders.extend(shopData)
        ttids.extend(ttidList)
    ttid_summary = []
    for ttid in ttids:
        ttid_stat = {}
        ttid_stat["ttid"] = ttid
        ttid_stat["time"] = dataTime
        ttid_stat["buy"] = []
        ttid_stat["sell"] = []
        ttid_stat["capacity"] = 0
        for order in orders:
            if order["ttid"] == ttid:
                ttid_stat["capacity"] += order["count"]
                if order["buy"] > 0:
                    ttid_stat["buy"].append(order["buy"] )
                if order["sell"] > 0:
                    ttid_stat["sell"].append(order["sell"] )  
        
        ttid_stat["buystat"] = {}
        if ttid_stat["buy"]:
            ttid_stat["buystat"]["avg"] = st.mean(ttid_stat["buy"])
            ttid_stat["buystat"]["min"] = min(ttid_stat["buy"])
            ttid_stat["buystat"]["max"] = max(ttid_stat["buy"])
            ttid_stat["buystat"]["gap"] = ttid_stat["buystat"]["max"] - ttid_stat["buystat"]["min"]
            ttid_stat["buystat"]["median"] = st.median(ttid_stat["buy"])
            ttid_stat["buystat"]["stdev"] = 0
            if len(ttid_stat["buy"]) >= 2:
                ttid_stat["buystat"]["stdev"] = st.stdev(ttid_stat["buy"], ttid_stat["buystat"]["avg"])
        else:
            ttid_stat["buystat"]["avg"] = 0.0
            ttid_stat["buystat"]["min"] = 0.0
            ttid_stat["buystat"]["max"] = 0.0
            ttid_stat["buystat"]["gap"] = 0.0
            ttid_stat["buystat"]["median"] = 0.0
            ttid_stat["buystat"]["stdev"] = 0.0
            
        ttid_stat["sellstat"] = {}
        if len(ttid_stat["sell"]) >= 2:
            ttid_stat["sellstat"]["avg"] = st.mean(ttid_stat["sell"])
            ttid_stat["sellstat"]["min"] = min(ttid_stat["sell"])
            ttid_stat["sellstat"]["max"] = max(ttid_stat["sell"])
            ttid_stat["sellstat"]["gap"] = ttid_stat["sellstat"]["max"] - ttid_stat["sellstat"]["min"]
            ttid_stat["sellstat"]["median"] = st.median(ttid_stat["sell"])
            ttid_stat["sellstat"]["stdev"] = 0
            if len(ttid_stat["sell"]) >= 2:
                ttid_stat["sellstat"]["stdev"] = st.stdev(ttid_stat["sell"], ttid_stat["sellstat"]["avg"])
        else:
            ttid_stat["sellstat"]["avg"] = 0.0
            ttid_stat["sellstat"]["min"] = 0.0
            ttid_stat["sellstat"]["max"] = 0.0
            ttid_stat["sellstat"]["median"] = 0.0
            ttid_stat["sellstat"]["gap"] = 0.0
            ttid_stat["sellstat"]["stdev"] = 0.0
        
        ttid_summary.append(ttid_stat)
        q = dbquery_insert.replace("\n", " ")
        try:
            cur.execute(q, (ttid_stat["time"],
                                            ttid_stat["ttid"],
                                            str(ttid_stat["buy"]),
                                            str(ttid_stat["sell"]),
                                            ttid_stat["capacity"],
                                            ttid_stat["buystat"]["avg"],
                                            ttid_stat["buystat"]["min"],
                                            ttid_stat["buystat"]["max"],
                                            ttid_stat["buystat"]["median"],
                                            ttid_stat["buystat"]["gap"],
                                            ttid_stat["buystat"]["stdev"],
                                            ttid_stat["sellstat"]["avg"],
                                            ttid_stat["sellstat"]["min"],
                                            ttid_stat["sellstat"]["max"],
                                            ttid_stat["sellstat"]["median"],
                                            ttid_stat["sellstat"]["gap"],
                                            ttid_stat["sellstat"]["stdev"]))
        except Exception as e:
            log.append("db exception %s in query %s" % (q, str(e)))
            return (None, fname, log)
    
    con.commit()
    con.close()
    sync()
    return (dbName, fname, log)

In [16]:
%%time
cleanupTmp()
sync()
full_pathes = [("%s/%s" % (os.getcwd(), f)) for f in data_sources]

def syncExec(pathes):
    parts = []
    counter = 0
    for p in pathes:
        print("%s - try (%d/%d)" % (p, counter, len(full_pathes)))
        parts.append(processSnapshot(p))
        print("%s - done (%d/%d)" % (p, counter, len(full_pathes)))
        sys.stdout.flush()
        counter += 1
    return parts
parts = workers.map_async(processSnapshot, full_pathes).get()
#parts = syncExec(full_pathes)
print("data size %s" % size(asizeof.asizeof(parts)))


clean tmp in /tmp
data size 1M
CPU times: user 128 ms, sys: 24.6 ms, total: 153 ms
Wall time: 38min 18s


In [17]:
#%%time
import shutil
def mprint(msg: str):
    print(msg)
    sys.stdout.flush()
   

outCon = sqlite3.connect(midDbTmp)
outcur = outCon.cursor()

backupDb = "%s/midData/mid_BU%d.db" % (os.getcwd(), mstime())
if os.path.isfile(midDbTmp):
    shutil.copyfile(midDbTmp, backupDb)
    sync()
mprint("midDbExample %s " % cutPathes(midDbExample))
mprint("midDb %s " % cutPathes(midDbTmp))
shutil.copyfile(midDbExample, midDbTmp)  
sync()

totalMerged = 0
partsCount = len(parts)
partsIndex = 0
for part in parts:
    partstart = time.time()
    if not part[0]:
        print("cannot process %s" % part[1])
        for logline in part[2]:
            print(logline)
        break
    dbName = part[0]
    mprint("process %s. (part %d/%d)" % (cutPathes(dbName), partsIndex, partsCount))
    secondDbName = 'db' + str(partsCount) + str(mstime())
    outcur.execute("ATTACH DATABASE ? as ? ;", (dbName, secondDbName))
    column_names="shapshotid,ttid,buy_list,sell_list,capacity,buystat_avg,buystat_min,buystat_max,buystat_median,buystat_gap,buystat_stdev,sellstat_avg,sellstat_min,sellstat_max,sellstat_median,sellstat_gap,sellstat_stdev"
    q = "INSERT INTO ttids (%s) SELECT %s FROM %s.ttids;" % (column_names, column_names, secondDbName)
    outcur.execute(q)
    outCon.commit()
    outcur.execute("DETACH  DATABASE %s ;" % secondDbName)
    partdone = time.time()
    partelapsed = partdone - partstart
    mprint("done -- %d parts merged (of %d). total %d. time %s " % (partsIndex, partsCount, totalMerged, str(partelapsed)))
    partsIndex += 1
    
outCon.commit()
q = "SELECT COUNT(ttid) FROM ttids"
outcur.execute(q)
print("done. records in mid db: %d" % outcur.fetchall()[0][0])
outCon.close()
shutil.copyfile(midDbTmp, midDb)

midDbExample midData/mid.example.db 
midDb /mid.1633797277323.db 
process /tmp/1628517800_shops.xml.1633797277556.db. (part 0/2751)
done -- 0 parts merged (of 2751). total 0. time 0.021094799041748047 
process /tmp/1628519402_shops.xml.1633797284282.db. (part 1/2751)
done -- 1 parts merged (of 2751). total 0. time 0.01388239860534668 
process /tmp/1628521201_shops.xml.1633797287246.db. (part 2/2751)
done -- 2 parts merged (of 2751). total 0. time 0.02862834930419922 
process /tmp/1628523001_shops.xml.1633797293354.db. (part 3/2751)
done -- 3 parts merged (of 2751). total 0. time 0.015137672424316406 
process /tmp/1628526601_shops.xml.1633797296343.db. (part 4/2751)
done -- 4 parts merged (of 2751). total 0. time 0.011030435562133789 
process /tmp/1628528401_shops.xml.1633797299743.db. (part 5/2751)
done -- 5 parts merged (of 2751). total 0. time 0.01516866683959961 
process /tmp/1628530201_shops.xml.1633797302562.db. (part 6/2751)
done -- 6 parts merged (of 2751). total 0. time 0.01875

done -- 59 parts merged (of 2751). total 0. time 0.008100032806396484 
process /tmp/1628629201_shops.xml.1633797421529.db. (part 60/2751)
done -- 60 parts merged (of 2751). total 0. time 0.007669687271118164 
process /tmp/1628631001_shops.xml.1633797423607.db. (part 61/2751)
done -- 61 parts merged (of 2751). total 0. time 0.008640766143798828 
process /tmp/1628632801_shops.xml.1633797425852.db. (part 62/2751)
done -- 62 parts merged (of 2751). total 0. time 0.006997585296630859 
process /tmp/1628634602_shops.xml.1633797427430.db. (part 63/2751)
done -- 63 parts merged (of 2751). total 0. time 0.0068149566650390625 
process /tmp/1628636401_shops.xml.1633797428984.db. (part 64/2751)
done -- 64 parts merged (of 2751). total 0. time 0.006462812423706055 
process /tmp/1628638201_shops.xml.1633797430391.db. (part 65/2751)
done -- 65 parts merged (of 2751). total 0. time 0.015535831451416016 
process /tmp/1628640001_shops.xml.1633797431810.db. (part 66/2751)
done -- 66 parts merged (of 2751)

process /tmp/1628735401_shops.xml.1633797544317.db. (part 119/2751)
done -- 119 parts merged (of 2751). total 0. time 0.012087583541870117 
process /tmp/1628737201_shops.xml.1633797545034.db. (part 120/2751)
done -- 120 parts merged (of 2751). total 0. time 0.004463911056518555 
process /tmp/1628739001_shops.xml.1633797545793.db. (part 121/2751)
done -- 121 parts merged (of 2751). total 0. time 0.011409521102905273 
process /tmp/1628742601_shops.xml.1633797546624.db. (part 122/2751)
done -- 122 parts merged (of 2751). total 0. time 0.008678913116455078 
process /tmp/1628744401_shops.xml.1633797547912.db. (part 123/2751)
done -- 123 parts merged (of 2751). total 0. time 0.01433563232421875 
process /tmp/1628746201_shops.xml.1633797549099.db. (part 124/2751)
done -- 124 parts merged (of 2751). total 0. time 0.015003681182861328 
process /tmp/1628748001_shops.xml.1633797551227.db. (part 125/2751)
done -- 125 parts merged (of 2751). total 0. time 0.008608818054199219 
process /tmp/16287498

process /tmp/1628847001_shops.xml.1633797670029.db. (part 178/2751)
done -- 178 parts merged (of 2751). total 0. time 0.010590076446533203 
process /tmp/1628848801_shops.xml.1633797672778.db. (part 179/2751)
done -- 179 parts merged (of 2751). total 0. time 0.012830734252929688 
process /tmp/1628850601_shops.xml.1633797675568.db. (part 180/2751)
done -- 180 parts merged (of 2751). total 0. time 0.014984369277954102 
process /tmp/1628852401_shops.xml.1633797678167.db. (part 181/2751)
done -- 181 parts merged (of 2751). total 0. time 0.013107776641845703 
process /tmp/1628854201_shops.xml.1633797680398.db. (part 182/2751)
done -- 182 parts merged (of 2751). total 0. time 0.008708715438842773 
process /tmp/1628856001_shops.xml.1633797682571.db. (part 183/2751)
done -- 183 parts merged (of 2751). total 0. time 0.01399540901184082 
process /tmp/1628857801_shops.xml.1633797685046.db. (part 184/2751)
done -- 184 parts merged (of 2751). total 0. time 0.00981593132019043 
process /tmp/162885960

process /tmp/1628955001_shops.xml.1633797867630.db. (part 237/2751)
done -- 237 parts merged (of 2751). total 0. time 0.017403841018676758 
process /tmp/1628958601_shops.xml.1633797872277.db. (part 238/2751)
done -- 238 parts merged (of 2751). total 0. time 0.009356260299682617 
process /tmp/1628960401_shops.xml.1633797875766.db. (part 239/2751)
done -- 239 parts merged (of 2751). total 0. time 0.01501154899597168 
process /tmp/1628962201_shops.xml.1633797878954.db. (part 240/2751)
done -- 240 parts merged (of 2751). total 0. time 0.015776395797729492 
process /tmp/1628964001_shops.xml.1633797883028.db. (part 241/2751)
done -- 241 parts merged (of 2751). total 0. time 0.012326240539550781 
process /tmp/1628965801_shops.xml.1633797886290.db. (part 242/2751)
done -- 242 parts merged (of 2751). total 0. time 0.011443614959716797 
process /tmp/1628967601_shops.xml.1633797889499.db. (part 243/2751)
done -- 243 parts merged (of 2751). total 0. time 0.009966373443603516 
process /tmp/16289694

...

process /tmp/1633023001_shops.xml.1633798196416.db. (part 2338/2751)
done -- 2338 parts merged (of 2751). total 0. time 0.01784491539001465 
process /tmp/1633024801_shops.xml.1633798203339.db. (part 2339/2751)
done -- 2339 parts merged (of 2751). total 0. time 0.017153501510620117 
process /tmp/1633026601_shops.xml.1633798210754.db. (part 2340/2751)
done -- 2340 parts merged (of 2751). total 0. time 0.013821125030517578 
process /tmp/1633028401_shops.xml.1633798217436.db. (part 2341/2751)
done -- 2341 parts merged (of 2751). total 0. time 0.014876604080200195 
process /tmp/1633030201_shops.xml.1633798225165.db. (part 2342/2751)
done -- 2342 parts merged (of 2751). total 0. time 0.014688730239868164 
process /tmp/1633032002_shops.xml.1633798232909.db. (part 2343/2751)
done -- 2343 parts merged (of 2751). total 0. time 0.03375649452209473 
process /tmp/1633033801_shops.xml.1633798240011.db. (part 2344/2751)
done -- 2344 parts merged (of 2751). total 0. time 0.011394500732421875 
process 

process /tmp/1633129201_shops.xml.1633798449105.db. (part 2396/2751)
done -- 2396 parts merged (of 2751). total 0. time 0.0092010498046875 
process /tmp/1633131001_shops.xml.1633798451698.db. (part 2397/2751)
done -- 2397 parts merged (of 2751). total 0. time 0.010738611221313477 
process /tmp/1633132801_shops.xml.1633798454483.db. (part 2398/2751)
done -- 2398 parts merged (of 2751). total 0. time 0.010040998458862305 
process /tmp/1633134601_shops.xml.1633798457477.db. (part 2399/2751)
done -- 2399 parts merged (of 2751). total 0. time 0.009040117263793945 
process /tmp/1633136401_shops.xml.1633798459680.db. (part 2400/2751)
done -- 2400 parts merged (of 2751). total 0. time 0.010101556777954102 
process /tmp/1633138201_shops.xml.1633798462178.db. (part 2401/2751)
done -- 2401 parts merged (of 2751). total 0. time 0.01847100257873535 
process /tmp/1633140001_shops.xml.1633798464274.db. (part 2402/2751)
done -- 2402 parts merged (of 2751). total 0. time 0.007534503936767578 
process /

process /tmp/1633235401_shops.xml.1633798638958.db. (part 2454/2751)
done -- 2454 parts merged (of 2751). total 0. time 0.007878780364990234 
process /tmp/1633237201_shops.xml.1633798641135.db. (part 2455/2751)
done -- 2455 parts merged (of 2751). total 0. time 0.008134603500366211 
process /tmp/1633239001_shops.xml.1633798643182.db. (part 2456/2751)
done -- 2456 parts merged (of 2751). total 0. time 0.008485078811645508 
process /tmp/1633240801_shops.xml.1633798644689.db. (part 2457/2751)
done -- 2457 parts merged (of 2751). total 0. time 0.02272963523864746 
process /tmp/1633242601_shops.xml.1633798646100.db. (part 2458/2751)
done -- 2458 parts merged (of 2751). total 0. time 0.010309219360351562 
process /tmp/1633244401_shops.xml.1633798649074.db. (part 2459/2751)
done -- 2459 parts merged (of 2751). total 0. time 0.013206243515014648 
process /tmp/1633246201_shops.xml.1633798651778.db. (part 2460/2751)
done -- 2460 parts merged (of 2751). total 0. time 0.014540672302246094 
process

process /tmp/1633341602_shops.xml.1633798824042.db. (part 2512/2751)
done -- 2512 parts merged (of 2751). total 0. time 0.016208648681640625 
process /tmp/1633343401_shops.xml.1633798829317.db. (part 2513/2751)
done -- 2513 parts merged (of 2751). total 0. time 0.017694473266601562 
process /tmp/1633345202_shops.xml.1633798834022.db. (part 2514/2751)
done -- 2514 parts merged (of 2751). total 0. time 0.021381378173828125 
process /tmp/1633347001_shops.xml.1633798838576.db. (part 2515/2751)
done -- 2515 parts merged (of 2751). total 0. time 0.022420167922973633 
process /tmp/1633348801_shops.xml.1633798842770.db. (part 2516/2751)
done -- 2516 parts merged (of 2751). total 0. time 0.015603065490722656 
process /tmp/1633350601_shops.xml.1633798847608.db. (part 2517/2751)
done -- 2517 parts merged (of 2751). total 0. time 0.015730857849121094 
process /tmp/1633352401_shops.xml.1633798853552.db. (part 2518/2751)
done -- 2518 parts merged (of 2751). total 0. time 0.011376619338989258 
proces

OperationalError: database or disk is full